In [ ]:
pip install mediapipe

In [ ]:
pip install pyautogui
pip install pydirectinput

In [1]:
import tensorflow as tf

tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6343938399440852165
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5742002176
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5171797598222879641
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:05:00.0, compute capability: 8.6"
]


In [14]:
# 버전 1
# 검지 : 직진
# 중지 : 후진
# 엄지 : 우회전
# 약지 : 좌회전


import cv2 as cv
import mediapipe as mp
import numpy as np
from pydirectinput import keyDown, keyUp, press

cap = cv.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


# Fingertip coordination
def tip_coord(num):
    finger = handmks.landmark[num*4]
    cv.putText(
        image, 
        f'{num}', # Text
        (round(finger.x*wi)-10, round(finger.y*hi)-10), # Org
        cv.FONT_ITALIC, # Font
        1, #Fonst Scale
        (0,0,0), # Font Color
        4 # Line Type
    ) 


up, back, right, left = 0, 0, 0, 0
stats_pre = [up, back, right, left]
cont = ['up', 'down', 'right', 'left']
    
    
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
    ) as hands:
    
    while cap.isOpened():                
        success, image = cap.read()
        hi, wi, ch = image.shape
        
        
        if not success: # 카메라 없을 경우
            print("No Camera")
            continue

        # Recolor image to RGB
        image.flags.writeable = False 
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB) 
        
        # Make detection
        results = hands.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        
        
        # Read Gesture
        if results.multi_hand_landmarks:
            for handmks in results.multi_hand_landmarks:
                
                text_x = (handmks.landmark[0].x * wi)
                text_y = (max([handmks.landmark[0].y, handmks.landmark[12].y]) * hi)
                
                # (오른쪽)손바닥이 위를 향할 때 / 새끼 손가락이 펴져있을 때 on
                if (handmks.landmark[5].x < handmks.landmark[17].x) and (handmks.landmark[19].y < handmks.landmark[20].y):
                     
                    
                    # forward
                    if handmks.landmark[8].y < handmks.landmark[6].y:
                        up = 1
                    else:
                        up = 0
                        
                    # back
                    if handmks.landmark[12].y < handmks.landmark[10].y:
                        back = 1
                    else:
                        back = 0

                    # right
                    if handmks.landmark[4].x > handmks.landmark[2].x:
                        right = 1
                    else:
                        right = 0
                    
                    # left
                    if (handmks.landmark[16].y < handmks.landmark[14].y):
                        left = 1
                    else:
                        left = 0
                        
                        
                    stats = [up, back, right, left]

                    # Control
                    if stats != stats_pre:
                        for stat in range(len(stats)):
                            if stats[stat] > stats_pre[stat]:
                                keyDown(cont[stat])
                            elif stats[stat] < stats_pre[stat]:
                                keyUp(cont[stat])

                    stats_pre = stats
                                
                    
                    
                    '''# on 상태 표시
                    status = 'on'
                    cv.putText(
                        image, 
                        status, 
                        (round(text_x) - 50, round(text_y) - 250), 
                        cv.FONT_HERSHEY_SIMPLEX, 
                        1, 
                        (80,50,40), 
                        4 
                    )'''
                    
                    # Render detections
                    mp_drawing.draw_landmarks(
                        image,
                        handmks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(190,120,110), thickness=2, circle_radius=2), # Color = BGR 
                        mp_drawing.DrawingSpec(color=(230,80,80), thickness=2, circle_radius=2)
                    )
                    
                else:
                    
                    # Render detections
                    mp_drawing.draw_landmarks(
                        image,
                        handmks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(120,125,255), thickness=2, circle_radius=2), # Color = BGR 
                        mp_drawing.DrawingSpec(color=(30,30,210), thickness=2, circle_radius=2)
                    )
                
                    '''# off 상태 표시
                    status = 'off'
                    cv.putText(
                        image, 
                        status, 
                        (round(text_x) - 50, round(text_y) - 250), 
                        cv.FONT_HERSHEY_SIMPLEX, 
                        1, 
                        (80,50,40), 
                        4 
                    ) '''


                '''
                # Fingertip mapping
                for i in range(1, 6):
                    tip_coord(i)
                '''
                                
                
        cv.imshow('MediaPipe Hands', image)
        
        if cv.waitKey(1) & 0xFF == ord('q'):
            for act in cont:
                keyUp(act)
            break
        

cap.release()
cv.destroyAllWindows()

In [6]:
# 버전 2
# 중지 :직진
# 검지 : 우회전
# 약지 : 좌회전
# 엄지 : Crtl


import cv2 as cv
import mediapipe as mp
import numpy as np
from pydirectinput import keyDown, keyUp, press

cap = cv.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


# Fingertip coordination
def tip_coord(num):
    finger = handmks.landmark[num*4]
    cv.putText(
        image, 
        f'{num}', # Text
        (round(finger.x*wi)-10, round(finger.y*hi)-10), # Org
        cv.FONT_ITALIC, # Font
        1, #Fonst Scale
        (0,0,0), # Font Color
        4 # Line Type
    ) 


up, right, left, ctr = 0, 0, 0, 0
stats_pre = [up, right, left, ctr]
cont = ['up', 'right', 'left', 'ctrl']
    
    
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
    ) as hands:
    
    while cap.isOpened():                
        success, image = cap.read()
        hi, wi, ch = image.shape
        
        
        if not success: # 카메라 없을 경우
            print("No Camera")
            continue

        # Recolor image to RGB
        image.flags.writeable = False 
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB) 
        
        # Make detection
        results = hands.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        
        
        # Read Gesture
        if results.multi_hand_landmarks:
            for handmks in results.multi_hand_landmarks:
                
                text_x = (handmks.landmark[0].x * wi)
                text_y = (max([handmks.landmark[0].y, handmks.landmark[12].y]) * hi)
                
                # (오른쪽)손바닥이 위를 향할 때 / 새끼 손가락이 펴져있을 때 on
                if (handmks.landmark[5].x < handmks.landmark[17].x) and (handmks.landmark[19].y < handmks.landmark[20].y):
                     
                    
                    # forward
                    if handmks.landmark[12].y < handmks.landmark[11].y:
                        up = 1
                    else:
                        up = 0

                    # right
                    if handmks.landmark[8].y < handmks.landmark[6].y:
                        right = 1
                    else:
                        right = 0
                    
                    # left
                    if (handmks.landmark[16].y < handmks.landmark[14].y):
                        left = 1
                    else:
                        left = 0
                        
                    # control
                    if (handmks.landmark[4].x > handmks.landmark[2].x):
                        ctr = 1
                    else:
                        ctr = 0
                        
                        
                    stats = [up, right, left, ctr]

                    # Control
                    if stats != stats_pre:
                        for stat in range(len(stats)):
                            if stats[stat] > stats_pre[stat]:
                                keyDown(cont[stat])
                            elif stats[stat] < stats_pre[stat]:
                                keyUp(cont[stat])

                    stats_pre = stats
                                
                    
                    
                    '''# on 상태 표시
                    status = 'on'
                    cv.putText(
                        image, 
                        status, 
                        (round(text_x) - 50, round(text_y) - 250), 
                        cv.FONT_HERSHEY_SIMPLEX, 
                        1, 
                        (80,50,40), 
                        4 
                    )'''
                    
                    # Render detections
                    mp_drawing.draw_landmarks(
                        image,
                        handmks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(190,120,110), thickness=2, circle_radius=2), # Color = BGR 
                        mp_drawing.DrawingSpec(color=(230,80,80), thickness=2, circle_radius=2)
                    )
                    
                else:
                    
                    # Render detections
                    mp_drawing.draw_landmarks(
                        image,
                        handmks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(120,125,255), thickness=2, circle_radius=2), # Color = BGR 
                        mp_drawing.DrawingSpec(color=(30,30,210), thickness=2, circle_radius=2)
                    )
                
                    '''# off 상태 표시
                    status = 'off'
                    cv.putText(
                        image, 
                        status, 
                        (round(text_x) - 50, round(text_y) - 250), 
                        cv.FONT_HERSHEY_SIMPLEX, 
                        1, 
                        (80,50,40), 
                        4 
                    ) '''


                '''
                # Fingertip mapping
                for i in range(1, 6):
                    tip_coord(i)
                '''
                                
                
        cv.imshow('MediaPipe Hands', image)
        
        if cv.waitKey(1) & 0xFF == ord('q'):
            for act in cont:
                keyUp(act)
            break
        

cap.release()
cv.destroyAllWindows()